In [1]:
%cd ../src

e:\00_CODE\03_Master_Thesis\rdf-literal-preprocessing\src


In [4]:
import fire
import sys
import tqdm
import pandas as pd
import numpy as np
import kgbench as kg
from collections import Counter
import matplotlib.pyplot as pl
import random as rd
from operator import itemgetter
import torch
import torch.nn.functional as F
from utils import RDF_NUMBER_TYPES, get_relevant_relations, add_triple, get_p_types, ALL_LITERALS
from kgbench.load import Data
from typing import List, Sequence, Tuple

from kgbench import load, tic, toc, d
import numpy as np
from sklearn.neighbors import LocalOutlierFactor
from utils import URI_PREFIX
import numpy as np
from sklearn.neighbors import LocalOutlierFactor
import datetime

In [2]:
from dataload import dmg777k

In [3]:
data = dmg777k()

loaded data dmg777k (55.27s).
pruned (9.95s).


In [6]:
relevent_relations = get_relevant_relations(data, relevant_types=RDF_NUMBER_TYPES)
num_bins=3
print(len(relevent_relations))
verbose = 1
i = 0
cumsum = 0
bins = np.arange(num_bins)
print("for rr in relrel")
    

3
for rr in relrel


In [7]:

print(num_bins)
for b in range(num_bins):
    o = (f'{URI_PREFIX}entity#binning{b+1}',f'{URI_PREFIX}datatype#bin')
    new_id = len(data.i2e)
    data.e2i[o] = new_id
    data.i2e.append(o)
    data.num_entities += 1
    if (verbose > 0):
        print(f'created new entity:')
        print(f'{data.e2i[o]} - {o}')



3
created new entity:
341271 - ('http://master-thesis.com/entity#binning1', 'http://master-thesis.com/datatype#bin')
created new entity:
341272 - ('http://master-thesis.com/entity#binning2', 'http://master-thesis.com/datatype#bin')
created new entity:
341273 - ('http://master-thesis.com/entity#binning3', 'http://master-thesis.com/datatype#bin')


In [8]:
print(relevent_relations)
for r in relevent_relations:
    p = f'{URI_PREFIX}predicat#binning{r}'
    new_id = len(data.i2r)
    data.r2i[p] = new_id
    data.i2r.append(p)
    data.num_relations += 1
    if (verbose > 0):
        print(f'created new relation:')
        print(f'{data.r2i[p]} - {p}')

[26, 2, 43]
created new relation:
60 - http://master-thesis.com/predicat#binning26
created new relation:
61 - http://master-thesis.com/predicat#binning2
created new relation:
62 - http://master-thesis.com/predicat#binning43


In [10]:
def encode_number_sublist(df: torch.Tensor, i2e: List[str]) -> torch.Tensor:
    sub_df = df.clone()
    for i in range(len(sub_df)):
        sub_df[i, 1] = torch.tensor(
            float(i2e[sub_df[i, 2]][0]), dtype=torch.float32)
    sub_df = sub_df[:, :2]
    return sub_df

In [29]:
print(len(data.triples))

777124


In [30]:
for rr in relevent_relations:
    print(rr)
    print(datetime.time())
    df = data.triples.clone()
    df = df[df[:, 1]== rr]
    sub_df = encode_number_sublist(df, data.i2e)
    print("sub2")
    sub_df = torch.cat( #put bins and sub_df together
        (sub_df, torch.from_numpy( #get numpy solutions back
            np.digitize( # assign for each value in sub_df the corresponding bin
                sub_df[:, 1], np.histogram( # calculate n bins based on values in sub_df
                    sub_df[:, 1], num_bins)[1][:-1]
                )
            ).reshape(-1, 1) # transfrom x tensor into (x,1) tensor to fit (x,2) shape of sub_df
        ), 1)
    
    ofn = lambda t: data.e2i[(f'{URI_PREFIX}entity#binning{t}',f'{URI_PREFIX}datatype#bin')]
    vofn = np.vectorize(ofn)
    pfn = lambda t: data.r2i[f'{URI_PREFIX}predicat#binning{rr}']
    vpfn = np.vectorize(pfn)
    to_add_df = sub_df.clone()
    print(len(to_add_df))
    #print(sub_df[:,2])
    print("aaa")
    to_add_df[:,1]= torch.tensor([vpfn(sub_df[:,2])], dtype=torch.int32)
    to_add_df[:,2]= torch.tensor([vofn(sub_df[:,2])],dtype=torch.int32)
    #print(to_add_df)
    #print(np.unique(vofn(sub_df[:,2])))
    #print(np.unique(vpfn(sub_df[:,2])))
    #print(vpfn(sub_df[:,2]))
    #print('eee')
    #print(num_bins)
    #print(torch.unique(sub_df[:,2]))
    #print("suib")
    data.triples = torch.cat((data.triples, to_add_df), 0)
    

26
00:00:00
sub2
510
aaa
2
00:00:00
sub2
8396
aaa
43
00:00:00
sub2
1800
aaa


In [31]:
print(len(data.triples))

787830


In [18]:
np.histogram( # calculate n bins based on values in sub_df
                    sub_df[:, 1], num_bins)[1][:-1]

array([4.50000e+01, 2.47242e+05, 4.94439e+05])

In [17]:
np.digitize( # assign for each value in sub_df the corresponding bin
                sub_df[:, 1], np.histogram( # calculate n bins based on values in sub_df
                    sub_df[:, 1], num_bins)[1][:-1]
                )

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
mport numpy as np
x = np.array([1, 2, 3, 4, 5])
squarer = lambda t: t ** 2
vfunc = np.vectorize(squarer)
vfunc(x)
# Output : array([ 1,  4,  9, 16, 25])